In [1]:
!pwd
!which python

/home/wwc/bem_git/bem/examples/SimpleTrap
/home/wwc/anaconda3/envs/ele36/bin/python


In [2]:
import sys
import logging, os
import numpy as np
# To import pyface.py is just for setting "Qt" sip API version to 2 before matplotlib sets it to default v1 
# that is incompatible with "pyside" in package "mayavi". wwc
import pyface.qt    
import matplotlib.pyplot as plt
from multiprocessing import Pool    # parallel computation.

sys.path.append('../../')   # add path of package "bem" to search list. wwc
sys.path.append('../../../electrode/')   # add path of package "electrode" to search list. wwc

In [3]:
# electrode is an another package in nist-ionstorage github. wwc
from electrode import System, GridElectrode

# load the electrostatics results into a electrode.System()
s = System()
for name in "DC1 DC2 DC3 DC4 DC5 RF".split():    #  DC2 DC3 DC4 DC5 RF
    # r = Result.from_vtk(prefix, name)
    # r after from_result(): e ~ obj = cls(origin=origin, spacing=spacing, data=pot). 
    # A vtk file after from_vtk(): obj = cls(origin=origin, spacing=spacing, data=pot). 
    # So after using from_vtk() we jump over r and get to e.
    e = GridElectrode.from_vtk("SimpleTrap_"+name+".vtk",maxderiv=4)    # from_vtk() is classmethod()
    e.name = name
    s.append(e)    # class System(list), System inherits list method "append" wwc
s["RF"].rf = 50.  # s["RF"] == e, create attribute e.rf  wwc
# For why instance s is subscriptable (can use []), this relates to "__getitem__() method" in System() class.  wwc
print(s["RF"],s["RF"].rf)
print(e,e.rf)

<electrode.electrode.GridElectrode object at 0x7f5231cafbf8> 50.0
<electrode.electrode.GridElectrode object at 0x7f5231cafbf8> 50.0


In [4]:
# # To understand the data structure of GridElectrode.data.  wwc
# print(e.origin, e.spacing)
# print(len(e.data),type(e.data))  # [pot_array,field_array,2nd_pot_array,3rd_pot_array], 0~3,4th derivative order.
# print(type(e.data[0]),type(e.data[1]))  # All list elements are ndarray.
# print(e.data[0].shape,e.data[1].shape)    # (nx,ny,nz,components)
# print(e.data[0])    # potential
# # [[[[p(x0,y0,z0)],
# #    [p(x0,y0,z1)],
# #    [p(x0,y0,z2)]],
# #    
# #   [[p(x0,y1,z0)],
# #    [p(x0,y1,z1)],
# #    [p(x0,y1,z2)]],     
# #
# #   ]]
# print(e.data[1])    # field
# # [[[[Ex(x0,y0,z0),Ey(x0,y0,z0),Ez(x0,y0,z0)],
# #    [Ex(x0,y0,z1),Ey(x0,y0,z1),Ez(x0,y0,z1)],
# #    [Ex(x0,y0,z2),Ey(x0,y0,z2),Ez(x0,y0,z2)]],
# #    
# #   [[Ex(x0,y1,z0),Ey(x0,y1,z0),Ez(x0,y1,z0)],
# #    [Ex(x0,y1,z1),Ey(x0,y1,z1),Ez(x0,y1,z1)],
# #    [Ex(x0,y1,z2),Ey(x0,y1,z2),Ez(x0,y1,z2)]],     
# #
# #   ]]

In [5]:
# This is from electrode turtorial.ipynb  wwc
# import scipy.constants as ct
# l = 30e-6 # µm length scale
# u = 20. # V rf peak voltage
# m = 25*ct.atomic_mass # ion mass
# q = 1*ct.elementary_charge # ion charge
# o = 2*np.pi*100e6 # rf frequency in rad/s
# s["RF"].rf = u*np.sqrt(q/m)/(2*l*o)
# print(s["RF"])

# x0 = s.minimum((0, 0, 1.), axis=(1, 2))

# for line in s.analyze_static(x0, axis=(1,0), m=m, q=q, l=l, o=o):
#     print(line)

In [6]:
from scipy.constants import atomic_mass
s["RF"].rf = 50.
s["DC1"].dc = 8
s["DC4"].dc = 8
s["DC5"].dc = 2
xi = (0, 0.5, 1.5)
x0 = s.minimum(xi)
print("offset: %s\n"%(x0-xi))
# analyze_static(x, axis=(0, 1, 2), m=ct.atomic_mass, q=ct.elementary_charge, l=100e-6, o=2*np.pi*1e6, ions=1, log=None):
for line in s.analyze_static(x0, m=25*atomic_mass, l=40e-6, o=100e6*2*np.pi):      # u=50., # u is V rf peak voltage, but it causes error.
    print(line)

offset: [ 0.00321592 -0.39990715 -0.60296574]

parameters:
 f=100 MHz, m=25 amu, q=1 qe, l=40 µm, scale=0.0391 V'/V_SI
coordinates:
 analyze point: [0.00321592 0.10009285 0.89703426]
               ([ 0.1286367   4.00371397 35.88137021] µm)
 minimum is at offset: [0. 0. 0.]
                      ([0. 0. 0.] µm)
potential:
 dc electrical: 1.4 eV
 rf pseudo: 0.00016 eV
 saddle offset: [2.36589110e-02 7.29276552e-05 8.51943581e-06]
               ([9.46356440e-01 2.91710621e-03 3.40777432e-04] µm)
 saddle height: -7.6e-07 eV
force:
 dc electrical: [ 0.00093877 -0.18838955  0.29004167] eV/l
               ([   23.46923978 -4709.73873163  7251.04169053] eV/m)
 rf pseudo: [-1.86489506e-05  1.76819955e-01 -2.88423429e-01] eV/l
           ([-4.66223766e-01  4.42049889e+03 -7.21058573e+03] eV/m)
modes:
 pp+dc normal curvatures: [3.03693252e-01 7.15252333e+02 7.19482547e+02]
 motion is bounded: False
 pseudopotential modes:
  a: 4.308 MHz, [9.99996228e-01 2.56067939e-03 9.93622846e-04]
  b: 209.